# 資料清洗
科系名稱裡面會出現排版用的空白字元 '\3000'

我也發現有些科系名稱裡面會有 0, [101年度] 此類字樣

有些學校會改名，比如2017年之後`新竹教育大學`的校名欄位變成`清華大學南大校區(原：新竹教育大學)`


# 資料來源
基本上只採集學士資料，為什麼?
因為在台灣碩士跟博士還有在職專班比較容易出現非應屆畢誒的問題
大部分的學士都是直接應屆高中上來(假設忽略重考)

In [1]:
import re
def countDuplicate(arr: [], removeSpace=True ):
    hs = dict()
    for val in arr:
        if isinstance(val,str) and removeSpace:
            val = "".join(val.split())
        hs[val] = hs.get(val,0) + 1
    return hs

def formatDataframeString(df, headNum=0):
    ''' clear all whitespaces in string fileds'''
    rows = ds.shape[0] if headNum<=0 else headNum
    cols = ds.shape[1]
    strColumns = [ i for i in ds.columns if isinstance(i,str)]
    for r in range(rows):
        for c in ds.columns:
            if isinstance(ds.at[r,c],str):
                ds.at[r,c] = "".join(ds.at[r,c].split())
    return df

# 資料匯入

In [2]:
import pandas as pd
fileName = './data/target.xlsx'

# only include the interested columns
ds = pd.read_excel('./data/target.xlsx')[['年度','學校名稱','科系代碼','科系名稱','等級別','總計','男生計','女生計']]
formatDataframeString(ds)


print("Dimension of {}: {}".format(fileName, ds.shape ) )
print("每列資料情況")
ds.head(15)

Dimension of ./data/target.xlsx: (109785, 8)
每列資料情況


,年度,學校名稱,科系代碼,科系名稱,等級別,總計,男生計,女生計
0,2018,政治大學,01111001,教育學系,博士,111,52,59
1,2018,政治大學,01111001,教育學系,碩士,45,14,31
2,2018,政治大學,01111001,教育學系,學士,239,71,168
3,2018,政治大學,01114005,教育行政與政策研究所,碩士,46,16,30
4,2018,政治大學,01114006,學校行政碩士在職專班,碩士,81,26,55
5,2018,政治大學,01121004,幼兒教育研究所,碩士,30,5,25
6,2018,政治大學,02211019,宗教研究所,博士,32,19,13
7,2018,政治大學,02211019,宗教研究所,碩士,28,12,16
8,2018,政治大學,02221020,歷史學系,博士,31,19,12
9,2018,政治大學,02221020,歷史學系,碩士,69,52,17


# 初步清洗資料(去除學位等級別)
要注意同一間學校下面相同科系代碼會對應到不同科系名稱
只取學士資料作分析。

轉換欄位為英文，比較好寫程式。

In [3]:
translated_name = {
    '年度':'year',
    '學校名稱': 'uniName',
    '科系代碼': 'depId',
    '科系名稱': 'depName',
    '等級別': 'degree',
    '總計': 'total',
    '男生計': 'male',
    '女生計': 'female'
}
ds.rename(columns = translated_name, inplace=True)

def filterExactCondition(df, dic):
    filteredTable = df
    for k,v in dic.items():
        filteredTable = filteredTable[ filteredTable[k] == v ]
    return filteredTable

bachelors = filterExactCondition(ds,{'degree':'學士'})
bachelors.tail(15)


,year,uniName,depId,depName,degree,total,male,female
99373,2009,台北市立教育大學,210401,音樂學系,學士,186,17,169
99376,2009,台北市立教育大學,210604,視覺藝術學系,學士,156,33,123
99381,2009,台北市立教育大學,220201,中國語文學系,學士,226,60,166
99384,2009,台北市立教育大學,220906,歷史與地理學系,學士,174,90,84
99386,2009,台北市立教育大學,310509,心理與諮商學系,學士,155,28,127
99388,2009,台北市立教育大學,310809,社會暨公共事務學系,學士,168,48,120
99390,2009,台北市立教育大學,520116,資訊科學系,學士,109,82,27
99398,2009,台北市立體育學院,210511,舞蹈系,學士,158,13,145
99399,2009,台北市立體育學院,210714,動態藝術學系,學士,120,64,56
99402,2009,台北市立體育學院,810305,球類運動學系,學士,501,359,142


# 每年各大學男女人數統計

In [133]:
from pprint import pprint
bachByYear = { i: filterExactCondition(bachelors,{'year':i}) for i in bachelors['year'].unique().tolist() }
#print(bachByYear[2017])

#print(filterExactCondition(ds,{'uniName':'中信金融管理學院'}))

uniStatByYear = {}

def getStatByYear(year):
    unis = {}
        #for uniName in bachelors['uniName'].unique().tolist():
    for uniName in bachByYear[year]['uniName'].unique().tolist():
        male, female,total = 0,0,0
        uni = {}
        for dep in filterExactCondition( bachByYear[year], {'uniName':uniName}).itertuples():
            male += getattr(dep, 'male')
            female += getattr(dep,'female')
            total += getattr(dep,'total')
            #print("hello")
        uni['total'] = total
        uni['male'] = male
        uni['female'] = female
        uni['uni'] = uniName
        uni['year'] =  year
        try:
            uni['ratio'] = uni['female'] / uni['male']
        except:
            uni['ratio'] = 0
        unis[uniName] = uni
    return unis

for y in bachelors['year'].unique().tolist():
    uniStatByYear[y] = getStatByYear(y)

#pprint(uniStatByYear[2016])


#### Save to ouptut
import pandas as pd
def getUniDictRatio(uniDict):
    return uniDict.get('ratio', 0)

def saveUniStatByYear():
    fileName = './output/uniStatByYear.xlsx'
    writer = pd.ExcelWriter(fileName,engine ='xlsxwriter')
    col_names = ['uni','ratio', 'total', 'male','female','year']
    newdf = pd.DataFrame(columns=col_names)
    for year in bachelors['year'].unique().tolist():
        for uni in sorted( uniStatByYear[year].values(), key=getUniDictRatio, reverse=True):
            newdf.loc[len(newdf)] = uni
        newdf.to_excel(writer, sheet_name=str(year))
    writer.save()
saveUniStatByYear()


# 研究資料
以前到現在女生比例最高的學校

In [108]:
from collections import OrderedDict

def getAndSetUniRatio(uniData):
    uniData['ratio'] = uniData['female']/uniData['male']
    return uniData['ratio']

uniStatByYear[2017]

# Top 5 female/male ratio within 10 years
statTop5Ratio = dict()
statBot5Ratio = dict()
for y in bachelors['year'].unique().tolist():
    ratioList =  sorted(uniStatByYear[y].values(), key=getAndSetUniRatio, reverse=True)
    #uniWithRatio = [ "{}({:.2f})".format(i['uni'], i['ratio']) for i in ratioList  ]
    uniWithRatio = [ i for i in ratioList  ]
    statTop5Ratio[y] = uniWithRatio[0:5]
    statBot5Ratio[y] = uniWithRatio[::-1][0:5]
    #pprint( "Year:{}, {}".format(y, [i for i in uniWithRatio]) )
    #pprint(uniWithBot5Ratio)
    
# == Two dict containing result    
#statTop5Ratio
#statBot5Ratio



{2018: [{'total': 4072,
   'male': 1220,
   'female': 2852,
   'uni': '台灣藝術大學',
   'year': 2018,
   'ratio': 2.3377049180327867},
  {'total': 774,
   'male': 240,
   'female': 534,
   'uni': '台南藝術大學',
   'year': 2018,
   'ratio': 2.225},
  {'total': 13291,
   'male': 4356,
   'female': 8935,
   'uni': '實踐大學',
   'year': 2018,
   'ratio': 2.0511937557392104},
  {'total': 3408,
   'male': 1174,
   'female': 2234,
   'uni': '台北教育大學',
   'year': 2018,
   'ratio': 1.9028960817717206},
  {'total': 1713,
   'male': 596,
   'female': 1117,
   'uni': '台北藝術大學',
   'year': 2018,
   'ratio': 1.8741610738255035}],
 2017: [{'total': 4017,
   'male': 1201,
   'female': 2816,
   'uni': '台灣藝術大學',
   'year': 2017,
   'ratio': 2.344712739383847},
  {'total': 784,
   'male': 235,
   'female': 549,
   'uni': '台南藝術大學',
   'year': 2017,
   'ratio': 2.3361702127659574},
  {'total': 13913,
   'male': 4626,
   'female': 9287,
   'uni': '實踐大學',
   'year': 2017,
   'ratio': 2.0075659316904453},
  {'total': 3383,


# 男女比前五名大學資料

In [55]:
from pyecharts import options as opts
from pyecharts.charts import Line
from pyecharts.globals import ThemeType

yearList = sorted( bachelors['year'].unique().tolist())

# get all universities ever enterd in the Top 5 Ratio
def getAllUniTop5List():
    allTop5 = set()
    for y in yearList:
        for uni in staticsTop5Ratio[y]:
            #print(uni['uni'])
            allTop5.add(uni['uni'])
    return allTop5

print("所有進入過男女比前五的學校:{}".format(getAllUniTop5List()))

def genRatiolist(uni, yearList, roundNum=2):
    nums = list()
    for y in yearList:
        
        num=None
        try:   
            num = round(uniStaticsByYear[y][uni]['ratio'],roundNum)
        except:
            num = None
        nums.append(num)
    return nums
    #return [ round(uniStaticsByYear[y][uni]['ratio'],roundNum) for y in yearList]
    
def getTop5RatioLineChart():
    line = (
        Line(init_opts=opts.InitOpts(
            theme= ThemeType.WESTEROS,
        ))
        .add_xaxis( [str(y) for y in yearList])
        .add_yaxis('台北教育大學',genRatiolist('台北教育大學', yearList), is_step= True)
        .add_yaxis('台灣藝術大學',genRatiolist('台灣藝術大學', yearList))
        .add_yaxis('法鼓文理學院',genRatiolist('法鼓文理學院', yearList))
        .add_yaxis('台南藝術大學',genRatiolist('台南藝術大學', yearList))
        .add_yaxis('台北市立教育大學',genRatiolist('台北市立教育大學', yearList))
        .add_yaxis('清華大學南大校區',genRatiolist('清華大學南大校區(原：新竹教育大學)', yearList))
        .add_yaxis('法鼓佛教學院',genRatiolist('法鼓佛教學院', yearList))
        .add_yaxis('台中教育大學',genRatiolist('台中教育大學', yearList))
        .add_yaxis('新竹教育大學',genRatiolist('新竹教育大學', yearList))
        .add_yaxis('實踐大學',genRatiolist('實踐大學', yearList))
        .add_yaxis('台北藝術大學',genRatiolist('台北藝術大學', yearList))
        .set_global_opts(title_opts=opts.TitleOpts(title='主標題',subtitle='副標題'))
    )
    return line

getTop5RatioLineChart().render_notebook()

所有進入過男女比前五的學校:{'台北教育大學', '台灣藝術大學', '法鼓文理學院', '台南藝術大學', '台北市立教育大學', '清華大學南大校區(原：新竹教育大學)', '法鼓佛教學院', '台中教育大學', '新竹教育大學', '實踐大學', '台北藝術大學'}


# 不同科系的男女比例排序
蒐集每個科系，每年的男女總人數。
只顯示人口總數超過100人的科系

In [130]:
from pprint import pprint
bachByYear = { i: filterExactCondition(bachelors,{'year':i}) for i in bachelors['year'].unique().tolist() }
#print(bachByYear[2017])

DepStatsByYear = {}
def getDepStatsByYear(year):
    deps = {}
        #for uniName in bachelors['uniName'].unique().tolist():
        
    for uniName in bachByYear[year]['uniName'].unique().tolist():
        for dep in filterExactCondition( bachByYear[year], {'uniName':uniName}).itertuples():
            #print(dep)
            depName = getattr(dep,'depName')
            dDic = deps.get( depName ,
                            dict({
                                'depName':depName,
                                'total':0,
                                'male':0,
                                'female':0,
                                'ratio':0,
                                'uniNames':[]
                            }))
            dDic['male'] += getattr(dep, 'male')
            dDic['female'] += getattr(dep,'female')
            dDic['total'] += getattr(dep,'total')
            # recompute every time, I know.
            ratio =0
            try:
                ratio = dDic['female'] / dDic['male']
            except:
                ratio = 0
            dDic['ratio'] = ratio
            if uniName not in dDic['uniNames']:
                dDic['uniNames'].append(uniName)
            deps[depName] = dDic
    return deps



def getDepDictRatio(depDict):
    return depDict.get('ratio', 0)

def getDepTotalGtN(depList,n):
    return [ dep for dep in depList if dep['total']>n ]

st = getDepStatsByYear(2014)
depRatioList =  sorted(getDepTotalGtN(st.values(), 6000), key=getDepDictRatio, reverse=True)
depRatioList
#for y in bachelors['year'].unique().tolist():
#    DepStatsByYear[y] = getDepStatsByYear(y)

#pprint(DepStatsByYear)





[{'depName': '會計學系',
  'total': 9267,
  'male': 3201,
  'female': 6066,
  'ratio': 1.895032802249297,
  'uniNames': ['政治大學',
   '台灣大學',
   '成功大學',
   '中興大學',
   '彰化師範大學',
   '台北大學',
   '東華大學',
   '屏東大學',
   '東海大學',
   '輔仁大學',
   '東吳大學',
   '中原大學',
   '中國文化大學',
   '逢甲大學',
   '靜宜大學',
   '義守大學',
   '銘傳大學',
   '實踐大學',
   '開南大學']},
 {'depName': '法律學系',
  'total': 7254,
  'male': 3299,
  'female': 3955,
  'ratio': 1.1988481357987268,
  'uniNames': ['政治大學',
   '台灣大學',
   '成功大學',
   '中正大學',
   '台北大學',
   '高雄大學',
   '東海大學',
   '輔仁大學',
   '東吳大學',
   '靜宜大學',
   '世新大學',
   '銘傳大學',
   '真理大學',
   '玄奘大學',
   '開南大學']},
 {'depName': '企業管理學系',
  'total': 14965,
  'male': 6923,
  'female': 8042,
  'ratio': 1.1616351292792142,
  'uniNames': ['政治大學',
   '台灣大學',
   '成功大學',
   '中興大學',
   '中央大學',
   '中山大學',
   '中正大學',
   '彰化師範大學',
   '台北大學',
   '嘉義大學',
   '東華大學',
   '金門大學',
   '屏東大學',
   '東海大學',
   '輔仁大學',
   '東吳大學',
   '中原大學',
   '中國文化大學',
   '逢甲大學',
   '靜宜大學',
   '中華大學',
   '大葉大學',
   '義守大學',
   '世新大學',
   

In [132]:
# save
#100, 1000, 4000, 6000, 10000

import pandas as pd

def saveDepRatioByYear_GTn(threshold=1000):
    fileName = './output/depRatioByYear_{}.xlsx'.format(threshold)
    #print(fileName)
    #return 
    writer = pd.ExcelWriter(fileName,engine ='xlsxwriter')
    
    col_names = ['depName','ratio', 'total', 'male','female','uniNames']
    newdf = pd.DataFrame(columns=col_names)
    for year in bachelors['year'].unique().tolist():
        depRatioList = sorted(getDepTotalGtN(getDepStatsByYear(year).values(), threshold),
                              key=getDepDictRatio,
                              reverse=True)
        for uni in depRatioList:
            #print(uni)
            newdf.loc[len(newdf)] = uni
        newdf.to_excel(writer, sheet_name=str(year))
    writer.save()
 
for i in [10000,6000,4000,1000,100]:
    saveDepRatioByYear_GTn(i)
#saveUniStatByYear()

# [研究] 科系代碼的意義
`科系代碼`數量 `科系名稱` 數量不同
看起來是一對多關係。
### 結論
2017年之後的科系代碼跟以前便制完全不一樣。
2017年之前的科系代碼跟以前完全相同

In [7]:
from pprint import pprint
print( "科系代碼數量: {}".format(len(countDuplicate(ds['depId']).keys())) )
print( "科系數量: {}".format(len(countDuplicate(ds['depName']).keys())))


def collapse( ds, toIdx, fromIdx, headNum=0 ):
    # 收集在 toIdx 欄位尚有相同值的row, 同時紀錄對應 fromIdx 欄位中的不重複資料
    if headNum <= 0:
        headNum = ds.shape[0] #total number of rows in ds
    todict = dict( [] )
    for index, row in ds.head(headNum).iterrows():
        arr = todict.get(row[toIdx], [])
        if row[fromIdx] not in arr:
            arr.append(row[fromIdx])
        todict[row[toIdx]] = arr
    return todict

#dID_dname = collapse(ds, 'depId', 'depName',1000)
#uni_dID = collapse(ds, 'uniName','depId', 1000)
#dID_uni = collapse(ds, 'depId','uniName',2000)
dname_uni = collapse(ds, 'depName','uniName')
a = dname_uni
for k,v in a.items():
    # 秀出來哪些學校有 "資訊工程學系" (完全符合)
    if len(v)>2 and k=="資訊工程學系":
        print("{}: {}".format(k,v))

科系代碼數量: 5001
科系數量: 10618
資訊工程學系: ['清華大學', '台灣大學', '台灣師範大學', '成功大學', '交通大學', '中央大學', '中山大學', '台灣海洋大學', '中正大學', '彰化師範大學', '台北大學', '嘉義大學', '高雄大學', '東華大學', '暨南國際大學', '台東大學', '宜蘭大學', '聯合大學', '台南大學', '台中教育大學', '金門大學', '屏東大學', '東海大學', '輔仁大學', '中原大學', '淡江大學', '中國文化大學', '逢甲大學', '靜宜大學', '長庚大學', '元智大學', '中華大學', '大葉大學', '義守大學', '銘傳大學', '南華大學', '真理大學', '大同大學', '長榮大學', '亞洲大學', '明新科技大學', '南榮科技大學', '健行科技大學', '正修科技大學', '元培科技大學', '景文科技大學', '虎尾科技大學', '清雲科技大學', '開南大學', '明道大學', '立德大學']


In [20]:
## 查看特定學校 資工系學士 歷年科系代碼變化
def filterExactCondition(df, dic):
    filteredTable = df
    for k,v in dic.items():
        filteredTable = filteredTable[ filteredTable[k] == v ]
    return filteredTable
filterDataFrame(ds, {'depName':'資訊工程學系', 'uniName':'中華大學', 'degree':'學士'})

,year,uniName,depId,depName,degree,total,male,female
5424,2018,中華大學,06131025,資訊工程學系,學士,391,343,48
15209,2017,中華大學,07141025,資訊工程學系,學士,386,334,52
34771,2015,中華大學,520114,資訊工程學系,學士,401,356,45
44446,2014,中華大學,520114,資訊工程學系,學士,393,341,52
54307,2013,中華大學,520114,資訊工程學系,學士,397,344,53
64293,2012,中華大學,520114,資訊工程學系,學士,434,372,62
74233,2011,中華大學,520114,資訊工程學系,學士,506,431,75
84273,2010,中華大學,520114,資訊工程學系,學士,614,526,88
94395,2009,中華大學,520114,資訊工程學系,學士,659,568,91
